In [7]:
def test_algorithm(algo, arms, num_sims, horizon):
    chosen_arms = [0.0 for i in range(num_sims * horizon)]
    rewards = [0.0 for i in range(num_sims * horizon)]
    cumulative_rewards = [0.0 for i in range(num_sims * horizon)]
    sim_nums = [0.0 for i in range(num_sims * horizon)]
    times = [0.0 for i in range(num_sims * horizon)]
  
    for sim in range(num_sims):
        sim = sim + 1
        algo.initialize(len(arms))
    
    for t in range(horizon):
        t = t + 1
        index = (sim - 1) * horizon + t - 1
        sim_nums[index] = sim
        times[index] = t
      
        chosen_arm = algo.select_arm()
        chosen_arms[index] = chosen_arm
      
        reward = arms[chosen_arms[index]].draw()
        rewards[index] = reward
      
        if t == 1:
            cumulative_rewards[index] = reward
        else:
            cumulative_rewards[index] = cumulative_rewards[index - 1] + reward
      
        algo.update(chosen_arm, reward)
  
    return [sim_nums, times, chosen_arms, rewards, cumulative_rewards]

In [8]:
import random

class BernoulliArm():
    def __init__(self, p):
        self.p = p
  
    def draw(self):
        if random.random() > self.p:
            return 0.0
        else:
            return 1.0

In [9]:
import math

def ind_max(x):
    m = max(x)
    return x.index(m)

class UCB1():
    def __init__(self, counts, values):
        self.counts = counts
        self.values = values
        return
  
    def initialize(self, n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
        return
  
    def select_arm(self):
        n_arms = len(self.counts)
        for arm in range(n_arms):
            if self.counts[arm] == 0:
                return arm

        ucb_values = [0.0 for arm in range(n_arms)]
        total_counts = sum(self.counts)
        for arm in range(n_arms):
            bonus = math.sqrt((2 * math.log(total_counts)) / float(self.counts[arm]))
            ucb_values[arm] = self.values[arm] + bonus
        return ind_max(ucb_values)
  
    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1
        n = self.counts[chosen_arm]

        value = self.values[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_arm] = new_value
        return

In [23]:
import random

random.seed(1)
means = [0.1, 0.1, 0.1, 0.1, 0.9]
n_arms = len(means)
random.shuffle(means)
arms = [BernoulliArm(mu) for mu in means]
# arms = map(lambda (mu): BernoulliArm(mu), means)
print("Best arm is " + str(ind_max(means)))

algo = UCB1([], [])
algo.initialize(n_arms)
results = test_algorithm(algo, arms, 5000, 250)

f = open("algorithms/ucb/ucb1_results.tsv", "w")

for i in range(len(results[0])):
    f.write("\t".join([str(results[j][i]) for j in range(len(results))]) + "\n")

f.close()

Best arm is 2


# DEV

In [24]:
import pandas as pd

In [25]:
column_names = ['sim_nums', 'times', 'chosen_arms', 'rewards', 'cumulative_rewards'] 
df = pd.read_csv('algorithms/ucb/ucb1_results.tsv', sep='\t', names=column_names)

In [26]:
df['sim_nums'].unique()

array([   0., 5000.])